<a href="https://colab.research.google.com/github/bobur-amirov/DataScienceExample/blob/main/aviachipta_narxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls

sample_data  sample_solution.csv  test_data.csv  train_data.csv


## Kerakli Kutubxonalarni yuklab olamiz

In [43]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [5]:
train_set = pd.read_csv('train_data.csv')
test_set = pd.read_csv('test_data.csv')

In [6]:
train_set.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [7]:
test_set.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,1,Air_India,AI-765,Kolkata,Evening,one,Night,Delhi,Business,28.25,2
1,2,Vistara,UK-747,Delhi,Early_Morning,one,Night,Mumbai,Business,13.83,34
2,3,Air_India,AI-570,Mumbai,Early_Morning,zero,Early_Morning,Chennai,Business,2.00,30
3,4,AirAsia,I5-974,Hyderabad,Night,one,Late_Night,Delhi,Economy,5.17,26
4,5,Air_India,AI-770,Kolkata,Night,one,Afternoon,Mumbai,Economy,16.33,35


In [8]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                20000 non-null  int64  
 1   airline           20000 non-null  object 
 2   flight            20000 non-null  object 
 3   source_city       20000 non-null  object 
 4   departure_time    20000 non-null  object 
 5   stops             20000 non-null  object 
 6   arrival_time      20000 non-null  object 
 7   destination_city  20000 non-null  object 
 8   class             20000 non-null  object 
 9   duration          20000 non-null  float64
 10  days_left         20000 non-null  int64  
 11  price             20000 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 1.8+ MB


In [9]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                5000 non-null   int64  
 1   airline           5000 non-null   object 
 2   flight            5000 non-null   object 
 3   source_city       5000 non-null   object 
 4   departure_time    5000 non-null   object 
 5   stops             5000 non-null   object 
 6   arrival_time      5000 non-null   object 
 7   destination_city  5000 non-null   object 
 8   class             5000 non-null   object 
 9   duration          5000 non-null   float64
 10  days_left         5000 non-null   int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 429.8+ KB


In [13]:
train_set['airline'].value_counts()

Vistara      8535
Air_India    5371
Indigo       2924
GO_FIRST     1508
AirAsia      1056
SpiceJet      606
Name: airline, dtype: int64

In [14]:
train_set['source_city'].value_counts()

Mumbai       4068
Delhi        4022
Bangalore    3486
Kolkata      3092
Hyderabad    2748
Chennai      2584
Name: source_city, dtype: int64

In [16]:
train_set['departure_time'].value_counts()

Morning          4738
Early_Morning    4432
Evening          4341
Night            3231
Afternoon        3166
Late_Night         92
Name: departure_time, dtype: int64

In [17]:
train_set['stops'].value_counts()

one            16666
zero            2440
two_or_more      894
Name: stops, dtype: int64

In [18]:
train_set['arrival_time'].value_counts()

Night            6142
Evening          5316
Morning          4086
Afternoon        2536
Early_Morning    1004
Late_Night        916
Name: arrival_time, dtype: int64

In [19]:
train_set['destination_city'].value_counts()

Mumbai       3918
Delhi        3825
Bangalore    3425
Kolkata      3323
Hyderabad    2814
Chennai      2695
Name: destination_city, dtype: int64

In [20]:
train_set['class'].value_counts()

Economy     13753
Business     6247
Name: class, dtype: int64

In [21]:
train_set.describe()

,id,duration,days_left,price
count,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,12.177627,25.924150,20960.281700
std,5773.647028,7.157944,13.624874,22775.459535
min,1.000000,0.830000,1.000000,1105.000000
25%,5000.750000,6.830000,14.000000,4783.000000
50%,10000.500000,11.250000,26.000000,7425.000000
75%,15000.250000,16.080000,38.000000,42521.000000
max,20000.000000,38.580000,49.000000,114523.000000


In [24]:
train_set.drop(['id', 'flight'], axis=1, inplace=True)
x_train = train_set.drop('price', axis=1)
y_data = train_set['price'].copy()

In [25]:
x_train.head()


,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,Vistara,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21
1,SpiceJet,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7
2,Vistara,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5
3,Vistara,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28
4,Indigo,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4


In [35]:
num_column = ['duration', 'days_left']


In [36]:
obj_column = ['airline', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class']

In [37]:
num_pipeline = Pipeline([
          ('std_scaler', StandardScaler())             
])
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_column),
    ('cat', OneHotEncoder(), obj_column)
])

In [38]:
x_data = full_pipeline.fit_transform(x_train)
x_data

<20000x37 sparse matrix of type '<class 'numpy.float64'>'
	with 180000 stored elements in Compressed Sparse Row format>

## ML qismi

In [39]:
LR_model = LinearRegression()
LR_model.fit(x_data, y_data)

LinearRegression()

In [40]:
x_test = test_set.drop(['id', 'flight'], axis=1)
x_test

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,Air_India,Kolkata,Evening,one,Night,Delhi,Business,28.25,2
1,Vistara,Delhi,Early_Morning,one,Night,Mumbai,Business,13.83,34
2,Air_India,Mumbai,Early_Morning,zero,Early_Morning,Chennai,Business,2.00,30
3,AirAsia,Hyderabad,Night,one,Late_Night,Delhi,Economy,5.17,26
4,Air_India,Kolkata,Night,one,Afternoon,Mumbai,Economy,16.33,35
...,...,...,...,...,...,...,...,...,...
4995,Air_India,Kolkata,Afternoon,one,Morning,Bangalore,Business,17.42,15
4996,Indigo,Kolkata,Morning,zero,Afternoon,Mumbai,Economy,3.00,40
4997,Air_India,Kolkata,Morning,one,Night,Mumbai,Business,11.17,37
4998,Air_India,Mumbai,Early_Morning,one,Night,Hyderabad,Business,13.33,38


In [41]:
x_data_test = full_pipeline.fit_transform(x_test)
x_data_test

<5000x37 sparse matrix of type '<class 'numpy.float64'>'
	with 45000 stored elements in Compressed Sparse Row format>

In [45]:
y_predicted = LR_model.predict(x_data_test)
y_predicted

array([55922.64459731, 53522.484482  , 40657.03904684, ...,
       51487.32651214, 48609.64181133, 51822.96773194])

In [50]:
y_ = y_data.to_numpy()
y_

array([ 7212,  5292, 60553, ..., 14822,  6450,  7141])

In [57]:
print('MAE LR_model =', mean_absolute_error(y_[:5000], y_predicted))
print('RMSE LR_model =', np.sqrt(mean_squared_error(y_[:5000], y_predicted)))

MAE LR_model = 23024.597431375336
RMSE LR_model = 31620.822159911728


In [52]:
submission = pd.DataFrame(y_predicted)
submission['id'] = test_set['id']
submission.rename({0:'price'},axis =1,inplace=True)
submission = submission[['id','price']]

In [53]:
submission.to_csv('submission.csv',index=False)

In [54]:
subm = pd.read_csv('submission.csv')
subm.head()

,id,price
0,1,55922.644597
1,2,53522.484482
2,3,40657.039047
3,4,2414.863961
4,5,5898.824224
